In [1]:
import micasense.imageset as imageset
import micasense.capture as capture
import os, glob
import multiprocessing
from pathlib import Path
import math
import numpy as np
import pandas as pd
import numpy as np
import datetime
import subprocess
from micasense import plotutils
import micasense.imageutils as imageutils
import matplotlib.pyplot as plt
import matplotlib.image as img
import skimage.measure
from ipywidgets import Tab, VBox, HBox, HTML, Output
import folium
from IPython.display import IFrame
import json
from shapely.geometry import Point, Polygon
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import pickle
import csv

######################################################################################################

def write_labeled_pixels(
        capture,
        blue, green, red, rededge, nir,
        wen, ndssi, ndwi,
        json_data,
        y_deres, x_deres,                # your processing deresolution
        output_csv_path,
        img_format=".png",
        label_res_y=4, 
        label_res_x=4
    ):
    """
    Fast labeled pixel extractor.
    Polygons come from labeling resolution (default 4x4).
    They are rescaled to your current resolution automatically.
    Bounding box scanning is used for speed.
    """

    uuid = capture.uuid
    json_filename = uuid + img_format

    # ------------------------------
    # 1. Find correct image entry
    # ------------------------------
    image_id = None
    for img in json_data["images"]:
        if img["file_name"] == json_filename:
            image_id = img["id"]
            break

    if image_id is None:
        return

    # ------------------------------
    # 2. Compute polygon scale factor
    # ------------------------------
    scale_x = x_deres / label_res_x
    scale_y = y_deres / label_res_y

    # ------------------------------
    # 3. Build scaled polygons
    # ------------------------------
    polygons = []
    for ann in json_data["annotations"]:
        if ann["image_id"] != image_id:
            continue

        pts = ann["segmentation"][0]
        scaled_coords = []

        for i in range(0, len(pts), 2):
            px = pts[i]   * scale_x
            py = pts[i+1] * scale_y
            scaled_coords.append((px, py))

        poly = Polygon(scaled_coords)
        minx, miny, maxx, maxy = poly.bounds

        polygons.append({
            "polygon": poly,
            "category_id": ann["category_id"],
            "bbox": (int(minx), int(miny), int(maxx), int(maxy))
        })

    if len(polygons) == 0:
        return

    H, W = blue.shape

    # ------------------------------
    # 4. Open CSV
    # ------------------------------
    new_file = not os.path.exists(output_csv_path)
    f = open(output_csv_path, "a", newline="")
    writer = csv.writer(f)

    if new_file:
        writer.writerow([
            "uuid","pixely","pixelx",
            "blue","green","red","rededge","nir",
            "wen","ndssi","ndwi",
            "class"
        ])

    # ------------------------------
    # 5. Bounding box pixel scanning
    # ------------------------------
    for p in polygons:

        poly   = p["polygon"]
        cat_id = p["category_id"]
        xmin, ymin, xmax, ymax = p["bbox"]

        # Clamp bounding box to image boundaries
        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(W-1, xmax)
        ymax = min(H-1, ymax)

        # Scan only inside bounding box
        for y in range(ymin, ymax + 1):
            for x in range(xmin, xmax + 1):

                pt = Point(x, y)
                if not (poly.contains(pt) or poly.touches(pt)):
                    continue

                # Pixel is inside polygon → write it
                writer.writerow([
                    uuid, y, x,
                    blue[y, x], green[y, x], red[y, x], rededge[y, x], nir[y, x],
                    wen[y, x], ndssi[y, x], ndwi[y, x],
                    cat_id
                ])

    f.close()



#####################################################################################################################
def extract_pixels_by_polygon_with_deresolution(csv_file_path, json_file_path, output_csv_path, downscale_factor=(1, 1), im_type='.jpg'):
    """
    Extract pixels from a downsampled CSV file that are bound by polygons from a JSON file generated by makesense.ai labeling tool.
    Adjusts for the resolution difference by scaling the pixel coordinates to match the original resolution.

    Parameters:
        csv_file_path (str): Path to the downsampled pixels.csv file containing pixel data.
        json_file_path (str): Path to the JSON file containing polygon labels and uuids.
        output_csv_path (str): Path to the output CSV file that will contain the filtered pixels.
        downscale_factor (tuple): The factor by which the resolution was reduced (e.g., (3, 3) for 3x3 blocks).
    """
    
    # Load the pixel data from the downsampled CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()  # Strip whitespace from column names
    
    # Ensure the dataframe contains the required columns
    if 'uuid' not in df.columns or 'pixely' not in df.columns or 'pixelx' not in df.columns:
        raise ValueError("The CSV must contain 'uuid', 'pixely', and 'pixelx' columns.")
    
    # Load the JSON file (labels from makesense.ai)
    with open(json_file_path) as f:
        label_data = json.load(f)
    
    # Extract the annotations and images from the JSON
    annotations = label_data['annotations']
    images_info = label_data['images']
    
    # Create a mapping of image ids to uuids (strip '.jpg' from filenames)
    image_id_to_uuid = {image['id']: image['file_name'].replace(im_type, '') for image in images_info}
    
    # Initialize an empty list to store filtered pixel data
    filtered_pixels = []
    
    # Scale factor for pixel coordinates to map back to original resolution
    scale_y, scale_x = downscale_factor
    
    # Iterate through each annotation (polygon)
    for annotation in annotations:
        image_id = annotation['image_id']
        image_uuid = image_id_to_uuid[image_id]  # Get the image UUID (filename)
        
        # Get the segmentation points (polygon) in the original resolution
        polygon_points = annotation['segmentation'][0]
        polygon_coords = [(polygon_points[i], polygon_points[i + 1]) for i in range(0, len(polygon_points), 2)]
        
        # Create a Polygon object using shapely
        polygon = Polygon(polygon_coords)
        
        # Filter the pixels from the CSV that correspond to this image's UUID
        image_pixels = df[df['uuid'] == image_uuid]
        
        # Check which pixels are inside the polygon in the original resolution
        for _, pixel_row in image_pixels.iterrows():
            # Scale up the coordinates to the original resolution
            original_x = pixel_row['pixelx'] * scale_x
            original_y = pixel_row['pixely'] * scale_y
            
            # Since the downsampled pixel represents a block, we check the block
            block_corners = [
                (original_x, original_y), 
                (original_x + scale_x - 1, original_y), 
                (original_x, original_y + scale_y - 1), 
                (original_x + scale_x - 1, original_y + scale_y - 1)
            ]
            
            # If any of the block corners fall inside the polygon, keep the pixel
            if any(polygon.contains(Point(corner)) for corner in block_corners):
                filtered_pixels.append(pixel_row)
    
    # Create a new DataFrame from the filtered pixels and save to CSV
    filtered_df = pd.DataFrame(filtered_pixels)
    filtered_df.to_csv(output_csv_path, index=False)
    
    print(f"Filtered pixels saved to {output_csv_path}")








#################################################################################################################
def im_reduction3d(im_aligned,y_div=1, x_div=1):
    #dimension of im_aligned is typically array is 1280 x 960

    new_array=skimage.measure.block_reduce(im_aligned,(y_div, x_div,1),np.mean)
    return new_array


#################################################################################################################
def im_reduction2d(im_aligned,y_div=1, x_div=1):
    #dimension of im_aligned is typically array is 1280 x 960

    new_array=skimage.measure.block_reduce(im_aligned,(y_div, x_div),np.mean)
    return new_array

####################################################################################################################
def acquire_bands(im_aligned,capture,imagePath,plot_outputs=0,save_plots=0):
    NIR_path= imagePath /'..'/ 'NIR'
    RED_path= imagePath /'..'/ 'RED'
    GREEN_path= imagePath /'..'/ 'GREEN'
    BLUE_path= imagePath /'..'/ 'BLUE'
    REDEDGE_path= imagePath /'..'/ 'REDEDGE'
    GREY= imagePath /'..'/ 'GREY'
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    
    blue_band = capture.band_names_lower().index('blue')
    green_band = capture.band_names_lower().index('green')
    red_band = capture.band_names_lower().index('red')
    nir_band = capture.band_names_lower().index('nir')
    redEdge_band = capture.band_names_lower().index('red edge')
    
    nir = im_aligned[:,:,capture.band_names_lower().index('nir')]
    red =im_aligned[:,:,capture.band_names_lower().index('red')]
    green=im_aligned[:,:,capture.band_names_lower().index('green')]
    blue=im_aligned[:,:,capture.band_names_lower().index('blue')]
    rededge=im_aligned[:,:,capture.band_names_lower().index('red edge')]
    
    
    le_file= capture.uuid +'.png'
    
    if not os.path.exists(GREY):
        os.makedirs(GREY)
        
    output_GREY = os.path.join(GREY, le_file)
    plt.imsave(output_GREY,green,cmap='gray')

    if plot_outputs==1:
        fig1, axis1 = plotutils.plotwithcolorbar(nir, 
                                                figsize = figsize, 
                                                title = f'NIR-{capture.uuid}-{capture.location()}',
                                                vmin = 0,
                                                vmax = 1)
        fig2, axis2 = plotutils.plotwithcolorbar(green, 
                                                figsize = figsize, 
                                                title = f'GREEN-{capture.uuid}-{capture.location()}',
                                                vmin = 0,
                                                vmax = 1)
        fig3, axis3 = plotutils.plotwithcolorbar(red, 
                                                figsize = figsize, 
                                                title = f'RED-{capture.uuid}-{capture.location()}',
                                                vmin = 0,
                                                vmax = 1)
        fig4, axis4 = plotutils.plotwithcolorbar(rededge, 
                                                figsize = figsize, 
                                                title = f'REDEDGE-{capture.uuid}-{capture.location()}',
                                                vmin = 0,
                                                vmax = 1)
        fig5, axis5 = plotutils.plotwithcolorbar(blue, 
                                                figsize = figsize, 
                                                title = f'BLUE-{capture.uuid}-{capture.location()}',
                                                vmin = 0,
                                                vmax = 1)
    
    
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(NIR_path):
            os.makedirs(NIR_path)
        output_NIR = os.path.join(NIR_path, le_file)
        fig1.savefig(output_NIR) 
        if not os.path.exists(RED_path):
            os.makedirs(RED_path)
        output_RED = os.path.join(RED_path, le_file)
        fig3.savefig(output_RED) 
        if not os.path.exists(GREEN_path):
            os.makedirs(GREEN_path)
        output_GREEN = os.path.join(GREEN_path, le_file)
        fig2.savefig(output_GREEN) 
        if not os.path.exists(BLUE_path):
            os.makedirs(BLUE_path)
        output_BLUE= os.path.join(BLUE_path, le_file)
        fig5.savefig(output_BLUE) 
        if not os.path.exists(REDEDGE_path):
            os.makedirs(REDEDGE_path)
        output_REDEDGE = os.path.join(REDEDGE_path, le_file)
        fig4.savefig(output_REDEDGE) 
            
    
        
    
    return nir,red,green,blue,rededge
############################################################################################################################
def NDVI_calculator(red,nir,capture,imagePath,v_min=-1,v_max=1,use_percentiles=0,plot_outputs=0,save_plots=0):
    
    NDVI_path= imagePath /'..'/ 'NDVI'
     
    

   

        
    #ignore division by zeros and calculate NDVI    
    np.seterr(divide='ignore', invalid='ignore')           
    ndvi = (nir - red) / (nir + red)
    
 
    if use_percentiles==1: 
        min_display_ndvi = np.percentile(ndvi.flatten(),v_min) # further mask soil by removing low-ndvi values
        max_display_ndvi = np.percentile(ndvi.flatten(), v_max)
    else:
        min_display_ndvi = v_min 
        max_display_ndvi = v_max
   


    #reduce the figure size to account for colorbar
    # figsize=(30,23) # use this size for full-image-resolution display
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    NDVI_file= capture.uuid +'.png'
    
    if plot_outputs:
        fig, axis = plotutils.plotwithcolorbar(ndvi, 
                                                figsize = figsize, 
                                                title = f'NDVI-{capture.uuid}-{capture.location()}',
                                                vmin = min_display_ndvi,
                                                vmax = max_display_ndvi)
    
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(NDVI_path):
            os.makedirs(NDVI_path)
        output_NDVI = os.path.join(NDVI_path, NDVI_file)
        fig.savefig(output_NDVI) 
    
      
    return ndvi



############################################################################################################
def greyscale_acquire(im_aligned,capture,imagePath):
    
    GREY= imagePath /'..'/ 'GREYSCALE'
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    green_band = capture.band_names_lower().index('green')
    green=im_aligned[:,:,capture.band_names_lower().index('green')]
    
    le_file= capture.uuid +'.png'
    
    if not os.path.exists(GREY):
        os.makedirs(GREY)    
    output_GREY = os.path.join(GREY, le_file)
    plt.imsave(output_GREY,green,cmap='gray')

    
###################################################################################################################   
def NDSSI_calculator(blue,nir,capture,imagePath,v_min=-1,v_max=1,use_percentiles=0,plot_outputs=1,save_plots=0):

    
    ndssi_path= imagePath /'..'/ 'ndssi'
     
    

   

        
    #ignore division by zeros and calculate NDVI    
    np.seterr(divide='ignore', invalid='ignore')           
    ndssi = (blue - nir) / (blue + nir)
    
 
    if use_percentiles==1: 
        min_display_ndssi = np.percentile(ndssi.flatten(),v_min) # further mask soil by removing low-ndvi values
        max_display_ndssi = np.percentile(ndssi.flatten(), v_max)
    else:
        min_display_ndssi = v_min 
        max_display_ndssi = v_max
   


    #reduce the figure size to account for colorbar
    # figsize=(30,23) # use this size for full-image-resolution display
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    ndssi_file= capture.uuid +'.png'
    
    if plot_outputs:
        fig, axis = plotutils.plotwithcolorbar(ndssi, 
                                                figsize = figsize, 
                                                title = f'ndssi-{capture.uuid}-{capture.location()}',
                                                vmin = min_display_ndssi,
                                                vmax = max_display_ndssi)
    
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(ndssi_path):
            os.makedirs(ndssi_path)
        output_ndssi = os.path.join(ndssi_path, ndssi_file)
        fig.savefig(output_ndssi) 
      
    return ndssi
    
##################################################################################################################
def wen_algo(red,green,capture,imagePath,v_min=-1,v_max=1,use_percentiles=0,plot_outputs=1,save_plots=0):
    #wen
    
    wen_path= imagePath /'..'/ 'wen_algo'
     
        
    #ignore division by zeros and calculate NDVI    
    np.seterr(divide='ignore', invalid='ignore')           
    wen = 3793.7 * np.power((green + red), 2) - 16.5

 
    if use_percentiles==1: 
        min_display_wen = np.percentile(wen.flatten(), v_min) # further mask soil by removing low-ndvi values
        max_display_wen = np.percentile(wen.flatten(), v_max)
    else:
        min_display_wen = v_min 
        max_display_wen = v_max
   


    #reduce the figure size to account for colorbar
    # figsize=(30,23) # use this size for full-image-resolution display
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    wen_file= capture.uuid +'.png'
    
    if plot_outputs:
        fig, axis = plotutils.plotwithcolorbar(wen, 
                                                figsize = figsize, 
                                                title = f'wen-{capture.uuid}-{capture.location()}',
                                                vmin = min_display_wen,
                                                vmax = max_display_wen)
    
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(wen_path):
            os.makedirs(wen_path)
        output_wen = os.path.join(wen_path, wen_file)
        fig.savefig(output_wen) 
      
    return wen

            
#####################################################################################################################
def NDWI_calculator(green,nir,capture,imagePath,v_min=-1,v_max=1,use_percentiles=0,plot_outputs=0,save_plots=0):

    
    NDWI_path= imagePath /'..'/ 'NDWI'
     
    

   

        
    #ignore division by zeros and calculate NDVI    
    np.seterr(divide='ignore', invalid='ignore')           
    ndwi = (green - nir) / (green + nir)
    
 
    if use_percentiles==1: 
        min_display_ndwi = np.percentile(ndwi.flatten(),v_min) # further mask soil by removing low-ndvi values
        max_display_ndwi = np.percentile(ndwi.flatten(), v_max)
    else:
        min_display_ndwi = v_min 
        max_display_ndwi = v_max
   


    #reduce the figure size to account for colorbar
    # figsize=(30,23) # use this size for full-image-resolution display
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    NDWI_file= capture.uuid +'.png'
    
    if plot_outputs:
        fig, axis = plotutils.plotwithcolorbar(ndwi, 
                                                figsize = figsize, 
                                                title = f'NDWI-{capture.uuid}-{capture.location()}',
                                                vmin = min_display_ndwi,
                                                vmax = max_display_ndwi)
    
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(NDWI_path):
            os.makedirs(NDWI_path)
        output_NDWI = os.path.join(NDWI_path, NDWI_file)
        fig.savefig(output_NDWI) 
      
    return ndwi
            

###################################################################################################################
def cir_composite(im_aligned,capture,imagePath):
    cir_band_indices = [capture.band_names_lower().index('nir'),
                    capture.band_names_lower().index('red'),
                    capture.band_names_lower().index('green')]
    
    CIR_path= imagePath /'..'/ 'CIR'
    

    

    if not os.path.exists(CIR_path):
        os.makedirs(CIR_path)
  
    
    im_display = np.zeros((im_aligned.shape[0],im_aligned.shape[1],im_aligned.shape[2]), dtype=np.float32 )
    
    #figsize=(30,23) # use this size for full-image-resolution display
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
 
    
    for i in cir_band_indices:
        im_display[:,:,i] =  imageutils.normalize(im_aligned[:,:,i])
        
    cir = im_display[:,:,cir_band_indices]
    

    CIR_file= capture.uuid +'.png'
    
    fig,axis=plotutils.plotwithcolorbar(cir,
                                          figsize=figsize,
                                          title = f'Color Infrared Composite-{capture.uuid}-{capture.location()}')
    
    output_CIR = os.path.join(CIR_path, CIR_file)
    
    fig.savefig(output_CIR)
    
  
    
    return cir
#####################################################################################################################


#################################################################################################################
def custom_signed_band_calculator(band1,band2,band3,band4,capture,imagePath,v_min=-1,v_max=1,use_percentiles=0,plot_outputs=0,save_plots=0):

    
    custom_path= imagePath /'..'/ 'Custom'
     
    if not os.path.exists(custom_path):
        os.makedirs(custom_path)

   

        
    #ignore division by zeros and calculate NDVI    
    np.seterr(divide='ignore', invalid='ignore')           
    custom = (band1 + band2) / (band3 + band4)
    
 
    if use_percentiles==1: 
        min_display_custom = np.percentile(custom.flatten(),v_min) # further mask soil by removing low-ndvi values
        max_display_custom = np.percentile(custom.flatten(), v_max)
    else:
        min_display_custom = v_min 
        max_display_custom = v_max
   


    #reduce the figure size to account for colorbar
    # figsize=(30,23) # use this size for full-image-resolution display
    
    figsize=(16,13)   # use this size for export-sized display
    figsize=np.asarray(figsize) - np.array([3,2])
    
    custom_file= capture.uuid +'.png'
    
    if plot_outputs:
        fig, axis = plotutils.plotwithcolorbar(custom, 
                                                figsize = figsize, 
                                                title = f'Custom-{capture.uuid}-{capture.location()}',
                                                vmin = min_display_custom,
                                                vmax = max_display_custom)
    if plot_outputs==1 & save_plots==1:
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        output_custom = os.path.join(custom_path, custom_file)
        fig.savefig(output_custom) 
      
    return custom
            
################################################################################################################
def capturepixellister(wen,ndssi,ndwi,blue,green,red,rededge,nir,capture,outputPath):
    
    
    if os.path.exists(os.path.join(outputPath,'pixels.csv')):
        lines=[]
    else:
        lines = ["SourceFile,uuid,\
        GPSDateStamp,GPSTimeStamp,\
        GPSLatitude,GPSLongitude,GPSAltitude,\
        pixely,pixelx,\
        wen,ndssi,ndwi,blue,green,red,rededge,nir,class\n"]
  
  
    #lines=[header]
    
    outputFilename = capture.uuid+'.tif'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    lat,lon,alt = capture.location()
    
    if os.path.exists(os.path.join(outputPath,'pixels.csv')):
        for i in range(ndwi.shape[0]):  # height
            for j in range(ndwi.shape[1]):  # width
                wen_pixel = wen[i, j]
                ndssi_pixel = ndssi[i, j]
                ndwi_pixel = ndwi[i, j]
                blue_pixel = blue[i, j]
                green_pixel = green[i, j]
                red_pixel = red[i, j]
                rededge_pixel = rededge[i, j]
                nir_pixel = nir[i, j]
  

                linestr = '"{}",{},'.format(fullOutputPath,capture.uuid)
                linestr += capture.utc_time().strftime("%Y:%m:%d,%H:%M:%S,")
                linestr += '{},{},{},'.format(lat,lon,alt)
                linestr += '{},{},'.format(i,j)
                linestr += '{},{},{},{},{},{},{},{},{}'.format(wen_pixel,ndssi_pixel,ndwi_pixel,blue_pixel,green_pixel,red_pixel,rededge_pixel,nir_pixel,custom_pixel)
                linestr += '\n' # when writing in text mode, the write command will convert to os.linesep
                lines.append(linestr)
            
    fullCsvPath = os.path.join(outputPath,'pixels.csv')
    with open(fullCsvPath, 'a') as csvfile: #create CSV
        csvfile.writelines(lines)



#####################################################################################################################


def analyze_pixel_data_from_csv(csv_file_path, band_ranges, height_sections):
    """
    Analyzes pixel data for multiple captures by parsing the entire CSV file, grouping data by the 'uuid' column,
    and generating plots and statistics for each band and section. The results are displayed in tabs.

    Parameters:
        csv_file_path (str): Path to the CSV file containing pixel data. There must be a 'uuid' column for each capture.
        band_ranges (dict): A dictionary where keys are band names (e.g., 'red', 'green') 
                            and values are tuples with major and minor ranges like: 
                            {'red': ((0.2, 1.0), (0.5, 0.7)), 'green': ((0.1, 0.9), (0.3, 0.6))}
        height_sections (int): Number of vertical sections to divide the image into.
        
    Returns:
        Displays a tab-based interface to traverse the results per capture (by 'uuid').
    """
    
    # Load the CSV and strip any leading/trailing spaces from column names
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()  # Strip whitespace from column names

    # Ensure the 'uuid' column exists
    if 'uuid' not in df.columns:
        raise ValueError("The CSV file must contain a 'uuid' column with capture names.")
    
    # Handle different possible column names for 'pixely'
    possible_pixely_columns = ['pixely', 'y', 'pixel_y', 'y_coord']
    pixely_col = next((col for col in possible_pixely_columns if col in df.columns), None)
    
    if pixely_col is None:
        raise ValueError("Could not find a suitable column for pixel y-coordinates ('pixely', 'y', 'pixel_y', etc.).")

    # Get unique capture uuids from the 'uuid' column
    capture_uuids = df['uuid'].unique()

    # Dictionary to store analysis results per capture
    results_per_capture = {}

    # Loop through each unique capture uuid
    for capture_uuid in capture_uuids:
        capture_df = df[df['uuid'] == capture_uuid].copy()  # Use `.copy()` to avoid setting with copy warning
        
        # Get image height using the correct y-coordinate column
        max_height = capture_df[pixely_col].max() + 1  # Use the identified column for y-coordinates
        section_height = max_height // height_sections
        capture_df.loc[:, 'height_section'] = capture_df[pixely_col] // section_height  # Fixing the SettingWithCopyWarning

        # Dictionary to store minor range percentages and HTML outputs for the capture
        capture_data = {
            'minor_range_percentages': {},
            'plots': []
        }

        # Loop through each band and its associated ranges
        for band, (major_range, minor_range) in band_ranges.items():
            # Check if the band_column exists
            if band not in capture_df.columns:
                raise ValueError(f"Band column '{band}' not found in the data.")
            
            # Store statistics and plots for each section
            for section in range(height_sections):
                section_pixel_values = capture_df[capture_df['height_section'] == section][band]

                # Count pixels in the major and minor range for this section
                major_pixels = section_pixel_values[(section_pixel_values >= major_range[0]) & (section_pixel_values <= major_range[1])]
                minor_pixels = section_pixel_values[(section_pixel_values >= minor_range[0]) & (section_pixel_values <= minor_range[1])]
                
                total_pixels_in_section = len(section_pixel_values)
                total_pixels_in_major = len(major_pixels)
                total_pixels_in_minor = len(minor_pixels)
                
                # Calculate the percentage of pixels in the minor range
                section_minor_percentage = (total_pixels_in_minor / total_pixels_in_major) * 100 if total_pixels_in_major > 0 else 0

                # Create the plot for this section
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.hist(section_pixel_values, bins=50, color='blue', edgecolor='black', alpha=0.7)
                ax.axvspan(minor_range[0], minor_range[1], color='red', alpha=0.3, label='Minor Range')
                ax.axvspan(major_range[0], major_range[1], color='green', alpha=0.3, label='Major Range')
                ax.set_title(f'{band.capitalize()} Pixel Value Distribution in Section {section+1} - {capture_uuid}')
                ax.set_xlabel(f'{band.capitalize()} Value')
                ax.set_ylabel('Number of Pixels')
                ax.legend()

                # Create an Output widget to capture the plot
                plot_output = Output()
                with plot_output:
                    plt.show(fig)
                plt.close(fig)  # Close the figure to avoid memory issues

                # HTML output for the stats displayed beside the plot
                stats_html = f"""
                <h4>Section {section+1} Stats</h4>
                <p>Total Pixels in Major Range: {total_pixels_in_major}</p>
                <p>Total Pixels in Minor Range: {total_pixels_in_minor}</p>
                <p>Minor Range Percentage: {section_minor_percentage:.2f}%</p>
                """

                # Create an HBox layout to put the plot and stats side by side
                plot_html = HBox([plot_output, HTML(stats_html)])
                capture_data['plots'].append(plot_html)

        # Save the data for this capture
        results_per_capture[capture_uuid] = capture_data
    
    # Create tabs to display each capture's data
    tab_children = []
    tab_titles = []

    for capture_uuid, data in results_per_capture.items():
        # Create a vertical layout to display the plots and stats for all sections
        tab_contents = VBox(data['plots'])
        tab_children.append(tab_contents)
        tab_titles.append(capture_uuid)

    # Create a tab widget
    tabs = Tab(children=tab_children)
    for idx, title in enumerate(tab_titles):
        tabs.set_title(idx, title)

    # Display the tabs in the notebook
    display(tabs)

# Example usage:
# band_ranges = {'red': ((0.2, 1.0), (0.5, 0.7)), 'green': ((0.1, 0.9), (0.3, 0.6))}
# analyze_pixel_data_from_csv('path_to_pixels.csv', band_ranges, 5)

    

####################################################################################################################
def masker(imagePath,csv_file_path, band_mask_ranges, image_shape, plot_outputs=0, save_plots=0, plot_overlay=0, save_overlay=0):
    """
    Masks pixels based on values in multiple bands from a CSV file (pixels.csv) for each unique capture (uuid).
    Uses 'pixelx' and 'pixely' to determine pixel positions for the mask. Optionally generates plots for the mask
    and an overlay of the mask on the original image.

    Parameters:
        csv_file_path (str): Path to the pixels.csv file containing pixel data.
        band_mask_ranges (dict): A dictionary where each key is a band (e.g., 'ndvi', 'ndwi') and the value is a tuple (m_min, m_max).
        image_shape (tuple): Shape of the image as (height, width) to reconstruct the mask correctly.
        plot_outputs (bool): If True, plots the mask.
        save_plots (bool): If True, saves the mask plot.
        plot_overlay (bool): If True, generates an overlay plot with the mask.
        save_overlay (bool): If True, saves the overlay plot.
        fullThumbnailPath (str): Path to the original thumbnail image for overlay.

    Returns:
        None (Masks and plots are generated for each unique capture in the CSV for each band).
    """
    
    # Load the pixel data from the CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()
    # Ensure the dataframe contains the required columns
    required_columns = ['uuid', 'pixely', 'pixelx']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The required column '{col}' was not found in the CSV.")
    
    # Ensure all band columns are in the dataframe
    for band in band_mask_ranges.keys():
        if band not in df.columns:
            raise ValueError(f"The band '{band}' was not found in the CSV.")
    
    # Get the height and width of the image
    height, width = image_shape
    
    
    # Iterate through each unique capture (uuid)
    for capture_uuid in df['uuid'].unique():
        # Filter data for this specific capture
        capture_df = df[df['uuid'] == capture_uuid]

        # Ensure the capture has data
        if capture_df.empty:
            print(f"No data found for capture UUID '{capture_uuid}'. Skipping...")
            continue

        # Iterate through each band and its corresponding mask range
        for band, (m_min, m_max) in band_mask_ranges.items():
            print(f"Processing band '{band}' for capture '{capture_uuid}' with range [{m_min}, {m_max}]")

            # Create an empty mask array for the entire image
            mask_array = np.zeros((height, width), dtype=np.float32)

            # Fill the mask array with values from the band based on pixelx, pixely positions
            for _, row in capture_df.iterrows():
                x = int(row['pixelx'])  # X coordinate
                y = int(row['pixely'])  # Y coordinate
                mask_array[y, x] = row[band]  # Set value in mask array

            # Apply the mask based on the specified range [m_min, m_max]
            masked = np.ma.masked_outside(mask_array, m_min, m_max)

            # Plot the mask if needed
            if plot_outputs == 1:
                plt.figure(figsize=(10, 6))
                plt.imshow(masked, cmap='coolwarm')
                plt.title(f'Masked {band} - {capture_uuid}')
                plt.colorbar()
                if save_plots == 1:
                    mask_path = imagePath /'..'/'MASK'
                    mask_path.mkdir(exist_ok=True)
                    plt.savefig(mask_path / f'{capture_uuid}_{band}_mask.png')
                plt.show()

            # Generate overlay if requested
            if plot_overlay == 1:
                GREY= imagePath /'..'/ 'GREY'
                GREY_file = capture_uuid+'.png'
                output_GREY = os.path.join(GREY, GREY_file)
                original_image = plt.imread(output_GREY)
                plt.figure(figsize=(10, 6))
                plt.imshow(original_image, alpha=0.7)
                plt.imshow(masked, cmap='coolwarm', alpha=0.3)
                plt.title(f'Overlayed Mask on {capture_uuid} ({band})')
                plt.colorbar()
                if save_overlay == 1:
                    overlay_path = imagePath /'..'/ 'OVERLAY'
                    overlay_path.mkdir(exist_ok=True)
                    plt.savefig(overlay_path / f'{capture_uuid}_{band}_overlay.png')
                plt.show()

    print("Masking completed for all captures and bands.")
############################################################################################################
def plot_metadata_csv(file_path,outputPath,save_map=0):
    # Read metadata csv
    metadata = pd.read_csv(file_path)
    
    
    # Calculate the mean of non-zero GPSLatitude and GPSLongitude values
    valid_latitudes = metadata['GPSLatitude'].dropna()
    valid_longitudes = metadata['GPSLongitude'].dropna()

    lat_mean = valid_latitudes[valid_latitudes != 0.0].mean()
    lon_mean = valid_longitudes[valid_longitudes != 0.0].mean()
    # Create a map with a basemap style (OpenStreetMap in this case)
    m = folium.Map(location=[lat_mean, lon_mean], zoom_start=10, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="ESRI")

    # Add points to the map
    for index, row in metadata.iterrows():
        if float(row['GPSLatitude']) == 0.0 and float(row['GPSLongitude']) == 0.0:
            folium.Marker([lat_mean, lon_mean], 
                          popup=f"SourceFile: {row['SourceFile']}\n"
                                 f"Date: {row['GPSDateStamp']}, Time: {row['GPSTimeStamp']}\n"
                                 f"Altitude: {row['GPSAltitude']}").add_to(m)
        else:
            folium.Marker([float(row['GPSLatitude']), float(row['GPSLongitude'])], 
                          popup=f"SourceFile: {row['SourceFile']}\n"
                                 f"Date: {row['GPSDateStamp']}, Time: {row['GPSTimeStamp']}\n"
                                 f"Altitude: {row['GPSAltitude']}").add_to(m)

    # Display the map in the notebook (using IFrame)
    
    #IFrame(src=m._to_html(), width=800, height=600)
    display(m)
    
    if save_map:
        m.save(outputPath/'metadata_map.html')
        
###############################################################################################################
####################################################################################################################        

def dnnmasker(imagePath, csv_file_path, model_path, plot_outputs=0, save_plots=0):
    """
    Applies a pre-trained machine learning model to predict pixel values (1/0 or others) based on band values.
    Uses 'pixelx' and 'pixely' to determine pixel positions and outputs the predictions.

    Parameters:
        imagePath (str or Path): Path to the directory where the images and predictions will be saved.
        csv_file_path (str): Path to the pixels.csv file containing pixel data, including GPS data.
        model_path (str): Path to the pre-trained model file (pickle format).
        plot_outputs (bool): If True, plots the prediction map.
        save_plots (bool): If True, saves the prediction plot.

    Returns:
        None (Predictions, plots, and logs are generated for each unique capture).
    """
    
    # Load the pixel data from the CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()

    # Ensure the dataframe contains the required columns
    required_columns = ['uuid', 'pixely', 'pixelx', 'blue', 'green', 'red', 'rededge', 'nir', 'GPSLatitude', 'GPSLongitude']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The required column '{col}' was not found in the CSV.")
    
    # Load the pre-trained machine learning model
    model = pickle.load(open(model_path, "rb"))

    # Determine image dimensions based on the maximum pixelx and pixely values
    height = df['pixely'].max() + 1  # +1 to account for zero-based indexing
    width = df['pixelx'].max() + 1



    # Prepare input data (features) for prediction for the whole dataset
    features = df[['ndvi','ndwi','blue', 'green', 'red', 'rededge', 'nir']].values
    
  

    # Make predictions for the entire dataset
    predictions = (model.predict(features)>0.5).astype('int32')

    # Add the predictions to the dataframe
    df['predictions'] = predictions

    # Create a list to store log data for the rflog CSV
    log_data = []

    # Iterate through each unique capture (uuid)
    for capture_uuid in df['uuid'].unique():
        # Filter data for this specific capture
        capture_df = df[df['uuid'] == capture_uuid]

        # Ensure the capture has data
        if capture_df.empty:
            print(f"No data found for capture UUID '{capture_uuid}'. Skipping...")
            continue

        # Extract GPS coordinates
        GPSLatitude = capture_df['GPSLatitude'].iloc[0]
        GPSLongitude = capture_df['GPSLongitude'].iloc[0]

        # Create an empty array to store predictions for the entire image
        prediction_array = np.zeros((height, width), dtype=np.float32)

        # Fill the prediction array with values based on pixelx, pixely positions
        for idx, row in capture_df.iterrows():
            x = int(row['pixelx'])  # X coordinate
            y = int(row['pixely'])  # Y coordinate

            # Ensure the pixel coordinates are within the calculated image bounds
            if 0 <= x < width and 0 <= y < height:
                prediction_array[y, x] = row['predictions']
            else:
                print(f"Warning: Pixel coordinates ({x}, {y}) are out of bounds for capture {capture_uuid}.")

        # Calculate the average prediction value for the capture
        avg_prediction = capture_df['predictions'].mean()

        # Plot the prediction map if needed
        if plot_outputs == 1:
            plt.figure(figsize=(10, 6))
            plt.imshow(prediction_array, cmap='coolwarm')
            plt.title(f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}')
            plt.colorbar()

            # Save the plot if required
            if save_plots == 1:
                prediction_path = imagePath / '..' / 'PREDICTIONS'
                prediction_path.mkdir(exist_ok=True, parents=True)
                filename = f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}.png'
                plt.savefig(prediction_path / filename)
            
            plt.show()

        # Append log data (capture name, GPS, and average prediction) for rflog
        log_data.append({
            'capture_uuid': capture_uuid,
            'GPSLatitude': GPSLatitude,
            'GPSLongitude': GPSLongitude,
            'avg_prediction': avg_prediction
        })

    # Save the log data as 'rflog.csv' in the same directory as the input CSV
    log_df = pd.DataFrame(log_data)
    log_csv_path = Path(csv_file_path).parent / 'ML_log.csv'
    log_df.to_csv(log_csv_path, index=False)

    print(f"Predictions completed for all captures. Log saved as {log_csv_path}.")
    
####################################################################################################################        

def probdnnmasker(imagePath, csv_file_path, model_path, plot_outputs=0, save_plots=0):
    """
    Applies a pre-trained machine learning model to predict pixel values (1/0 or others) based on band values.
    Uses 'pixelx' and 'pixely' to determine pixel positions and outputs the predictions.

    Parameters:
        imagePath (str or Path): Path to the directory where the images and predictions will be saved.
        csv_file_path (str): Path to the pixels.csv file containing pixel data, including GPS data.
        model_path (str): Path to the pre-trained model file (pickle format).
        plot_outputs (bool): If True, plots the prediction map.
        save_plots (bool): If True, saves the prediction plot.

    Returns:
        None (Predictions, plots, and logs are generated for each unique capture).
    """
    
    # Load the pixel data from the CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()

    # Ensure the dataframe contains the required columns
    required_columns = ['uuid', 'pixely', 'pixelx', 'blue', 'green', 'red', 'rededge', 'nir', 'GPSLatitude', 'GPSLongitude']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The required column '{col}' was not found in the CSV.")
    
    # Load the pre-trained machine learning model
    model = pickle.load(open(model_path, "rb"))

    # Determine image dimensions based on the maximum pixelx and pixely values
    height = df['pixely'].max() + 1  # +1 to account for zero-based indexing
    width = df['pixelx'].max() + 1



    # Prepare input data (features) for prediction for the whole dataset
    features = df[['ndvi','ndwi','blue', 'green', 'red', 'rededge', 'nir']].values
    
  

    # Make predictions for the entire dataset
    predictions = model.predict(features)
  


    # Add the predictions to the dataframe
    df['predictions'] = predictions

    # Create a list to store log data for the rflog CSV
    log_data = []

    # Iterate through each unique capture (uuid)
    for capture_uuid in df['uuid'].unique():
        # Filter data for this specific capture
        capture_df = df[df['uuid'] == capture_uuid]

        # Ensure the capture has data
        if capture_df.empty:
            print(f"No data found for capture UUID '{capture_uuid}'. Skipping...")
            continue

        # Extract GPS coordinates
        GPSLatitude = capture_df['GPSLatitude'].iloc[0]
        GPSLongitude = capture_df['GPSLongitude'].iloc[0]

        # Create an empty array to store predictions for the entire image
        prediction_array = np.zeros((height, width), dtype=np.float32)

        # Fill the prediction array with values based on pixelx, pixely positions
        for idx, row in capture_df.iterrows():
            x = int(row['pixelx'])  # X coordinate
            y = int(row['pixely'])  # Y coordinate

            # Ensure the pixel coordinates are within the calculated image bounds
            if 0 <= x < width and 0 <= y < height:
                prediction_array[y, x] = row['predictions']
            else:
                print(f"Warning: Pixel coordinates ({x}, {y}) are out of bounds for capture {capture_uuid}.")

        # Calculate the average prediction value for the capture
        avg_prediction = capture_df['predictions'].mean()

        # Plot the prediction map if needed
        if plot_outputs == 1:
            plt.figure(figsize=(10, 6))
            plt.imshow(prediction_array, cmap='coolwarm')
            plt.title(f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}')
            plt.colorbar()

            # Save the plot if required
            if save_plots == 1:
                prediction_path = imagePath / '..' / 'PROBPREDICTIONS'
                prediction_path.mkdir(exist_ok=True, parents=True)
                filename = f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}.png'
                plt.savefig(prediction_path / filename)
            
            plt.show()

        # Append log data (capture name, GPS, and average prediction) for rflog
        log_data.append({
            'capture_uuid': capture_uuid,
            'GPSLatitude': GPSLatitude,
            'GPSLongitude': GPSLongitude,
            'avg_prediction': avg_prediction
        })

    # Save the log data as 'rflog.csv' in the same directory as the input CSV
    log_df = pd.DataFrame(log_data)
    log_csv_path = Path(csv_file_path).parent / 'ML_prob_log.csv'
    log_df.to_csv(log_csv_path, index=False)

    print(f"Predictions completed for all captures. Log saved as {log_csv_path}.")

##############################################################################################################
def rfmasker(imagePath, csv_file_path, model_path, plot_outputs=0, save_plots=0):
    """
    Applies a pre-trained machine learning model to predict pixel values (1/0 or others) based on band values.
    Uses 'pixelx' and 'pixely' to determine pixel positions and outputs the predictions.

    Parameters:
        imagePath (str or Path): Path to the directory where the images and predictions will be saved.
        csv_file_path (str): Path to the pixels.csv file containing pixel data, including GPS data.
        model_path (str): Path to the pre-trained model file (pickle format).
        plot_outputs (bool): If True, plots the prediction map.
        save_plots (bool): If True, saves the prediction plot.

    Returns:
        None (Predictions, plots, and logs are generated for each unique capture).
    """
    
    # Load the pixel data from the CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()

    # Ensure the dataframe contains the required columns
    required_columns = ['uuid', 'pixely', 'pixelx', 'blue', 'green', 'red', 'rededge', 'nir', 'GPSLatitude', 'GPSLongitude']
    #for col in required_columns:
        #if col not in df.columns:
           # raise ValueError(f"The required column '{col}' was not found in the CSV.")
    
    # Load the pre-trained machine learning model
    model = pickle.load(open(model_path, "rb"))

    # Determine image dimensions based on the maximum pixelx and pixely values
    height = df['pixely'].max() + 1  # +1 to account for zero-based indexing
    width = df['pixelx'].max() + 1



    # Prepare input data (features) for prediction for the whole dataset
    features = df[['ndvi','ndwi','blue', 'green', 'red', 'rededge', 'nir']].values
    
  

    # Make predictions for the entire dataset
    predictions = model.predict(features)

    # Add the predictions to the dataframe
    df['predictions'] = predictions

    # Create a list to store log data for the rflog CSV
    log_data = []

    # Iterate through each unique capture (uuid)
    for capture_uuid in df['uuid'].unique():
        # Filter data for this specific capture
        capture_df = df[df['uuid'] == capture_uuid]

        # Ensure the capture has data
        if capture_df.empty:
            print(f"No data found for capture UUID '{capture_uuid}'. Skipping...")
            continue

        # Extract GPS coordinates
        GPSLatitude = capture_df['GPSLatitude'].iloc[0]
        GPSLongitude = capture_df['GPSLongitude'].iloc[0]

        # Create an empty array to store predictions for the entire image
        prediction_array = np.zeros((height, width), dtype=np.float32)

        # Fill the prediction array with values based on pixelx, pixely positions
        for idx, row in capture_df.iterrows():
            x = int(row['pixelx'])  # X coordinate
            y = int(row['pixely'])  # Y coordinate

            # Ensure the pixel coordinates are within the calculated image bounds
            if 0 <= x < width and 0 <= y < height:
                prediction_array[y, x] = row['predictions']
            else:
                print(f"Warning: Pixel coordinates ({x}, {y}) are out of bounds for capture {capture_uuid}.")

        # Calculate the average prediction value for the capture
        avg_prediction = capture_df['predictions'].mean()

        # Plot the prediction map if needed
        if plot_outputs == 1:
            plt.figure(figsize=(10, 6))
            plt.imshow(prediction_array, cmap='coolwarm')
            plt.title(f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}')
            plt.colorbar()

            # Save the plot if required
            if save_plots == 1:
                prediction_path = imagePath / '..' / 'PREDICTIONS'
                prediction_path.mkdir(exist_ok=True, parents=True)
                filename = f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}.png'
                plt.savefig(prediction_path / filename)
            
            plt.show()

        # Append log data (capture name, GPS, and average prediction) for rflog
        log_data.append({
            'capture_uuid': capture_uuid,
            'GPSLatitude': GPSLatitude,
            'GPSLongitude': GPSLongitude,
            'avg_prediction': avg_prediction
        })

    # Save the log data as 'rflog.csv' in the same directory as the input CSV
    log_df = pd.DataFrame(log_data)
    log_csv_path = Path(csv_file_path).parent / 'ML_log.csv'
    log_df.to_csv(log_csv_path, index=False)

    print(f"Predictions completed for all captures. Log saved as {log_csv_path}.")
    
####################################################################################################################        

def probrfmasker(imagePath, csv_file_path, model_path, plot_outputs=0, save_plots=0):
    """
    Applies a pre-trained machine learning model to predict pixel values (1/0 or others) based on band values.
    Uses 'pixelx' and 'pixely' to determine pixel positions and outputs the predictions.

    Parameters:
        imagePath (str or Path): Path to the directory where the images and predictions will be saved.
        csv_file_path (str): Path to the pixels.csv file containing pixel data, including GPS data.
        model_path (str): Path to the pre-trained model file (pickle format).
        plot_outputs (bool): If True, plots the prediction map.
        save_plots (bool): If True, saves the prediction plot.

    Returns:
        None (Predictions, plots, and logs are generated for each unique capture).
    """
    
    # Load the pixel data from the CSV file
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()

    # Ensure the dataframe contains the required columns
    required_columns = ['uuid', 'pixely', 'pixelx', 'blue', 'green', 'red', 'rededge', 'nir', 'GPSLatitude', 'GPSLongitude']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The required column '{col}' was not found in the CSV.")
    
    # Load the pre-trained machine learning model
    model = pickle.load(open(model_path, "rb"))

    # Determine image dimensions based on the maximum pixelx and pixely values
    height = df['pixely'].max() + 1  # +1 to account for zero-based indexing
    width = df['pixelx'].max() + 1



    # Prepare input data (features) for prediction for the whole dataset
    features = df[['ndvi','ndwi','blue', 'green', 'red', 'rededge', 'nir']].values
    
  

    # Make predictions for the entire dataset
    predictions = model.predict_proba(features)
  


    # Add the predictions to the dataframe
    df['predictions'] = predictions[:,1]

    # Create a list to store log data for the rflog CSV
    log_data = []

    # Iterate through each unique capture (uuid)
    for capture_uuid in df['uuid'].unique():
        # Filter data for this specific capture
        capture_df = df[df['uuid'] == capture_uuid]

        # Ensure the capture has data
        if capture_df.empty:
            print(f"No data found for capture UUID '{capture_uuid}'. Skipping...")
            continue

        # Extract GPS coordinates
        GPSLatitude = capture_df['GPSLatitude'].iloc[0]
        GPSLongitude = capture_df['GPSLongitude'].iloc[0]

        # Create an empty array to store predictions for the entire image
        prediction_array = np.zeros((height, width), dtype=np.float32)

        # Fill the prediction array with values based on pixelx, pixely positions
        for idx, row in capture_df.iterrows():
            x = int(row['pixelx'])  # X coordinate
            y = int(row['pixely'])  # Y coordinate

            # Ensure the pixel coordinates are within the calculated image bounds
            if 0 <= x < width and 0 <= y < height:
                prediction_array[y, x] = row['predictions']
            else:
                print(f"Warning: Pixel coordinates ({x}, {y}) are out of bounds for capture {capture_uuid}.")

        # Calculate the average prediction value for the capture
        avg_prediction = capture_df['predictions'].mean()

        # Plot the prediction map if needed
        if plot_outputs == 1:
            plt.figure(figsize=(10, 6))
            plt.imshow(prediction_array, cmap='coolwarm')
            plt.title(f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}')
            plt.colorbar()

            # Save the plot if required
            if save_plots == 1:
                prediction_path = imagePath / '..' / 'PROBPREDICTIONS'
                prediction_path.mkdir(exist_ok=True, parents=True)
                filename = f'{capture_uuid}_lat{GPSLatitude}_lon{GPSLongitude}_avg{avg_prediction:.2f}.png'
                plt.savefig(prediction_path / filename)
            
            plt.show()

        # Append log data (capture name, GPS, and average prediction) for rflog
        log_data.append({
            'capture_uuid': capture_uuid,
            'GPSLatitude': GPSLatitude,
            'GPSLongitude': GPSLongitude,
            'avg_prediction': avg_prediction
        })

    # Save the log data as 'rflog.csv' in the same directory as the input CSV
    log_df = pd.DataFrame(log_data)
    log_csv_path = Path(csv_file_path).parent / 'ML_prob_log.csv'
    log_df.to_csv(log_csv_path, index=False)

    print(f"Predictions completed for all captures. Log saved as {log_csv_path}.")
